In [12]:
# Import the libraries
import keras_nlp
import tensorflow_datasets as tfds
import tensorflow as tf

In [20]:
# Hyperparameters
seq_len = 128
batch_size = 8
preset = "gpt2_base_en"
epochs = 5

In [5]:
# Load the preprocessor
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(preset, sequence_length=seq_len)

In [6]:
# Load the model 
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(preset, preprocessor=preprocessor)

In [10]:
# Sample a generation output
output = gpt2_lm.generate("The dark wizard was ", max_length=seq_len)
print(output)

The dark wizard was  a wizard that was not a wizard. He was an evil wizard. He was a dark wizard who was not evil but a dark wizard that was evil. He was a dark wizard who was a dark wizard. He was dark wizard that was evil. He is an evil wizard and he is evil wizard. He is evil wizard that is evil wizard.
The dark wizard's name is  Maggie.
He has dark magic.
Maggie has dark magic.
The dark wizard has dark magic.
Maggie has dark magic.
The dark wizard has dark magic.


In [16]:
# Load a custom dataset
ds = tfds.load("reddit_tifu", split="train", shuffle_files=True, as_supervised=True)

# Sample the dataset
for x, y in ds.take(1):
    print("INPUT: ", x.numpy().decode("utf-8"))
    print("OUTPUT: ", y.numpy().decode("utf-8"))

INPUT:  me and a friend decided to go to the beach last sunday. we loaded up and headed out. we were about half way there when i decided that i was not leaving till i had seafood. 

now i'm not talking about red lobster. no friends i'm talking about a low country boil. i found the restaurant and got directions. i don't know if any of you have heard about the crab shack on tybee island but let me tell you it's worth it. 

we arrived and was seated quickly. we decided to get a seafood sampler for two and split it. the waitress bought it out on separate platters for us. the amount of food was staggering. two types of crab, shrimp, mussels, crawfish, andouille sausage, red potatoes, and corn on the cob. i managed to finish it and some of my friends crawfish and mussels. it was a day to be a fat ass. we finished paid for our food and headed to the beach. 

funny thing about seafood. it runs through me faster than a kenyan 

we arrived and walked around a bit. it was about 45min since we arr

2023-06-05 20:50:39.161385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [2]
	 [[{{node Placeholder/_1}}]]
2023-06-05 20:50:39.161774: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2]
	 [[{{node Placeholder/_0}}]]


In [19]:
# Preprocess the dataset
train_ds = (
    ds.map(lambda document, _: document)
    .batch(batch_size)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [21]:
# Sample the dataset
# train_ds = train_ds.take(500)

In [22]:
# Learning rate schedule
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate = 5e-5,
    decay_steps = train_ds.cardinality().numpy() * epochs,
    end_learning_rate = 0.0
)

In [25]:
# Loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [26]:
# Compile the model
gpt2_lm.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

In [27]:
# Train the model
gpt2_lm.fit(train_ds, epochs=epochs, verbose=1)

2023-06-05 21:12:02.023381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_42' with dtype int32
	 [[{{node Placeholder/_42}}]]
2023-06-05 21:12:02.024011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [2]
	 [[{{node Placeholder/_4}}]]


Epoch 1/5


2023-06-05 21:12:18.079528: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. gpt2_causal_lm/gpt2_backbone/embeddings_dropout/dropout/random_uniform/RandomUniform
2023-06-05 21:12:19.885614: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2023-06-05 21:12:37.483466: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



9803/9968 [============================>.] - ETA: 42s - loss: 3.1417 - accuracy: 0.3476

KeyboardInterrupt: 

In [28]:
# Sample a generation output
output = gpt2_lm.generate("The dark wizard was ", max_length=seq_len)
print(output)

2023-06-05 21:55:04.363601: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



The dark wizard was  

a man with a beard who was  

in the middle of an argument  

i asked him if he wanted to go to the bar and he said he wanted to go to the bar.  

i asked if he wanted to go to the bar with him.  

i asked if i would be happy to go with him, he said no i would be happy to go with him.  

i asked if he wanted to go to the bar with me. he said no i would like to go with him. i asked if


In [30]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("The dark wizard was ", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("The dark wizard was ", max_length=200)
print("\nGPT-2 output:")
print(output)

2023-06-05 21:55:56.627060: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.




GPT-2 output:
The dark wizard was                                                                                                                           



GPT-2 output:
The dark wizard was 

a man who had been in a relationship for a few years. 

he was a very nice guy, but he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick. 

he was a bit of a prick.
